## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [27]:
import tflearn
import numpy as np

import os.path as osp
import tensorflow as tf
from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder, unpickle_data

from latent_3d_points.src.tf_utils import reset_tf_graph

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [49]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.


model_dir = osp.join(top_out_dir, 'single_class_ae_chair_chamfer')
experiment_name = 'single_class_ae_chair_chamfer'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'chamfer'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = 'chair'

Load Point-Clouds

In [50]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)    # e.g. /home/yz6/code/latent_3d_points/data/shape_net_core_uniform_samples_2048/02691156
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=1, file_ending='.ply', verbose=True)

6778 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed here). For more details please use print, etc.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of this many epochs)
    'saver_step': 10       (# how many epochs to save neural-network)

In [51]:

train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [52]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            train_dir = train_dir,
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).

In [53]:
reset_tf_graph()
print tf.get_default_graph().as_graph_def()

versions {
  producer: 24
}



Build AE Model.

In [54]:

ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
> /home/yz6/code/latent_3d_points/src/encoders_decoders.py(46)encoder_with_convs_and_symmetry()
-> if verbose:
(Pdb) c
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_chair_chamfer_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

> /home/yz6/code/latent_3d_points/src/encoders_decoders.py(45)encoder_with_convs_and_symmetry()
-> pdb.set_trace()
(Pdb) c
encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_chair_chamfer_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

> /home/yz6/code/latent_3d_points/src/encoders_decoders.py(46)encoder_with_convs_and_symmetry()
-> if verbose:
(Pdb) c
encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_chair_chamfer_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

> /home/yz6/code/latent_3d_points/src/encoders_decoders.py(45)encoder_with_convs_and_symmetry()
-> pdb.set_trace

In [55]:
model_dir

'../data/single_class_ae_chair_chamfer'

Train the AE (save output to train_stats.txt) 

In [61]:
ae.restore_model(model_dir, 500)

INFO:tensorflow:Restoring parameters from ../data/single_class_ae_chair_chamfer/models.ckpt-500


In [64]:

train_pc = list(unpickle_data('/home/yz6/code/latent_3d_points/data/single_class_ae_chair_chamfer/train_pc.pkl'))
train_pc = train_pc[0]
n_examples = train_pc.num_examples
batch_size = conf.batch_size
n_batches = int(n_examples / batch_size)
latent_list=list()
for _ in xrange(n_batches):
    feed_pc, feed_model_names, _ = train_pc.next_batch(batch_size)
    latent_codes = ae.transform(feed_pc)
    latent_list.append(latent_codes)
    
latent = np.concatenate(latent_list, axis=0)
# feed_pc, _, _ = train_pc.full_epoch_data(shuffle=False)
# latent = ae.transform(feed_pc)
np.save(osp.join(model_dir, 'train_hidden.npy'), latent)

Get some reconstuctions and latent-codes.

In [ ]:


reconstructions = ae.reconstruct(feed_pc)


In [65]:
latent.shape

(5750, 128)

In [66]:
train_pc.num_examples

5761